<table style="width: 100%; border-style: none;">
<tr style="border-style: none">
<td style="border-style: none; width: 1%; font-size: 16px">Institut f&uuml;r Theoretische Physik<br /> Universit&auml;t zu K&ouml;ln</td>
<td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Prof. Dr. Simon Trebst<br />MSc. Carsten Bauer</td>
</tr>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Computerphysik</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Vorlesung &mdash; Programmiertechniken 3</h1>
<hr>
<h3 style="font-weight:bold; text-align: center; margin: 0px; padding:0px; margin-bottom: 20px;">Sommersemester 2020</h3>

**Website:** [http://www.thp.uni-koeln.de/trebst/Lectures/2020-CompPhys.shtml](http://www.thp.uni-koeln.de/trebst/Lectures/2020-CompPhys.shtml)

**Themen dieses Notebooks:** Plots erstellen, Sortieren (BubbleSort), Timing und Komplexität

## Plots erstellen

In [ ]:
plot(x_values, f.(x_values), label="quadratic")
plot(x_values, exp.(x_values), label="exponential")
legend()
xlabel("time")
ylabel("amount")

## Sortieren

### Kurze Übung: Elemente in einem Array vertauschen

In Julia gibt es die Konvention, dass Funktionen die mindestens eines ihrer Funktionsargumente modifizieren, mit einem Ausrufezeichen am Ende versehen werden.

In [ ]:
function swap!(a, i, j)
    tmp = a[i]
    a[i] = a[j]
    a[j] = tmp
    return a
end

### Selbst sortieren: BubbleSort

**Bubble Schritt**

"Die Zahl 10 ist wie eine Blase (Bubble) im Array aufgestiegen", d.h. ans Ende gewandert.

Wir könnten diesen Vorgang nun so oft wiederholen, bis das Array sortiert ist. Da wir allerdings schon wissen, dass die Größte Zahl bereits am Ende steht reicht es das Teilarray mit den Elementen `a[1]` bis `a[length(a) - 1]` zu sortieren.

Diese Logik führt uns zum [Bubble Sort](https://de.wikipedia.org/wiki/Bubblesort) Algorithmus.

#### Bubble Sort Algorithmus

In [ ]:
function bubblesort!(a)
    N = length(a)
    
    for last in N:-1:2
        for i in 1:(last-1)
            if a[i] > a[i+1] # wenn links größer als rechts
                swap!(a, i, i+1)
            end
        end
    end
    
    return a
end

### Visualisierung

In [ ]:
# Der folgende Code ist kein Vorlesungsinhalt!
using PyPlot, Random

function show_bubble_schritt(n)
    a = shuffle(1:n)
    
    pygui(true)
    fig = figure()
    title("Bubble-Schritt")
    for last = length(a):-1:length(a)
        # bubble-Schritt
        for i in 1:last-1
            if a[i]>a[i+1]
                swap!(a, i, i+1)
            end
            fig.clear()
            bar(1:length(a), a)
            m, mind = findmax(a)
            bar(mind, m, color="red")
            sleep(0.001)
        end
    end
    pygui(false)
    nothing
end

function show_bubble_sort(n)
    a = shuffle(1:n)
    
    pygui(true)
    fig = figure()
    title("Bubble-Sort")
    for last = length(a):-1:2
        # bubble-Schritt
        for i in 1:last-1
            if a[i]>a[i+1]
                swap!(a, i, i+1)
            end
        end
        fig.clear()
        bar(1:length(a), a)
        m, mind = findmax(a[1:last])
        bar(mind, m, color="red")
        sleep(0.001)
    end
    pygui(false)
    nothing
end

In [ ]:
show_bubble_schritt(40)

In [ ]:
show_bubble_sort(40)

## Timing und Komplexität

### Zeitmessung

In [ ]:
b = rand(50000);

In [ ]:
@time bubblesort!(b);

In [ ]:
function benchmark_bubblesort()
    number_count = [0.0]
    elapsed_time = [0.0]

    for i in 1:16
        b = rand(2^i)
        t = @elapsed bubblesort!(b)
        println(2^i, "\t", t)
        push!(number_count, 2^i)
        push!(elapsed_time, t)
    end
    
    return number_count, elapsed_time
end

In [ ]:
number_count, elapsed_time = benchmark_bubblesort();

In [ ]:
plot(number_count, elapsed_time, marker="o", label="Bubblesort");
legend()
xlabel("Länge des Arrays (n)");
ylabel("Zeit (t)");
title("Bubble Sort Zeitmessung")

Frage: Wie skaliert die Bubble Sort Laufzeit mit der Array Länge $n$?

### Polynomieller Fit auf einer Log-Log-Skala

In [ ]:
using Polynomials

# fit straight line in loglog space (ignoring first couple of datapoints)
# Syntax: polyfit(x,y, polynomial_degree)
p = polyfit(log.(number_count[7:end]), log.(elapsed_time[7:end]), 1)
m = p.a[2] # Steigung

In [ ]:
plot(number_count, elapsed_time, marker="o", label="Bubblesort");
plot(number_count, exp.(p.(log.(number_count))), label="Fit (Steigung ≈ $(round(m, digits=2)))")
legend();
xscale("log")
yscale("log")
xlabel("Länge des Arrays (n)");
ylabel("Zeit (t)");
title("Bubble Sort Zeitmessung")

**Komplexität (asymptotisches Verhalten):** BubbleSort $\in \mathcal{O}(n^2)$

O-Notation: https://de.wikipedia.org/wiki/Landau-Symbole#Beispiele_und_Notation

### Vergleich mit Julia's `sort!`

In [ ]:
function benchmark_juliasort()
    number_count = [0.0]
    elapsed_time = [0.0]

    for i in 1:16
        b = rand(2^i)
        t = @elapsed sort!(b)
        push!(number_count, 2^i)
        push!(elapsed_time, t)
    end
    
    return number_count, elapsed_time
end

number_count, elapsed_time = benchmark_juliasort();

# fit straight line in loglog space (ignoring first couple of datapoints)
p = polyfit(log.(number_count[7:end]), log.(elapsed_time[7:end]), 1)
m = p.a[2]

plot(number_count, elapsed_time, marker="o", label="sort!");
plot(number_count, exp.(p.(log.(number_count))), label="Fit (Steigung ≈ $(round(m, digits=2)))")
legend(loc=4);
xscale("log")
yscale("log")
xlabel("Länge des Arrays (n)");
ylabel("Zeit (t)");
title("Julia's sort! Zeitmessung");

Übersicht der Komplexität verschiedener Sortierverfahren: https://de.wikipedia.org/wiki/Sortierverfahren

## Randnotiz zu Zeitmessungen: BenchmarkTools.jl

Randnotiz: Die Macros `@time` und `@elapsed` sind hilfreich, sollten jedoch meistens vermieden werden, da Nebeneffekte das Messergebnis verzerren können. Führen Sie beispielsweise `@time sort(rand(1000));` zweimal aus und beobachten Sie, wie sich das Ergebnis ändert.

Es ist stattdessen empfehlenswert `@btime` und `@belapsed` aus dem Paket [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl) zu verwenden.

Demonstration:

In [ ]:
@time sort(rand(1000));

In [ ]:
@time sort(rand(1000));

In [ ]:
using BenchmarkTools

In [ ]:
@btime sort(rand(1000));

In [ ]:
@btime sort(rand(1000));